In [1]:
import binascii

In [2]:
def get_binary_str(hex_str):
    out = bin(int(hex_str, 16))[2:]
    padding = 4 - (len(out) % 4)
    if padding != 0:
        out = '0' * padding + out
    assert len(out) % 4 == 0
    return out

expected_out = '00111000000000000110111101000101001010010001001000000000'
get_binary_str('38006F45291200') == expected_out

True

In [3]:
get_binary_str('38006F45291200') == '00111000000000000110111101000101001010010001001000000000'

True

In [4]:
packet = '38006F45291200'
binary_str = get_binary_str(packet)
print(binary_str)
main_version = binary_str[:3]
print(main_version)
packet_type = binary_str[3:6]
print(packet_type)
if packet_type == '100':
    print('literal packet! no other versions')
else:
    length_type_id = binary_str[6]
    print(length_type_id)
    if length_type_id == '0':
        # next 15 bits are a number that represents the total length in bits
        subpacket_length = int(binary_str[7:22], 2)
        print(subpacket_length)
        subpacket_str = binary_str[22:22+subpacket_length]
        print(subpacket_str)

00111000000000000110111101000101001010010001001000000000
001
110
0
27
110100010100101001000100100


In [5]:
def parse_packet(binary_str):
    main_version = binary_str[:3]
    packet_type = binary_str[3:6]
    if packet_type == '100':
        length_type_id = None
        payload = binary_str[6:]
        for i in range(len(payload)):
            if i % 5 == 0 and possible_payload[i] == '0':
                # if this bit is 0, this is the last group
                payload = payload[:i+5]
                return (main_version, packet_type, length_type_id, payload)
                
    else:
        length_type_id = binary_str[6]
        payload = binary_str[7:]
        if length_type_id == '0':
            # next 15 bits are a number that represents the total subpackets length in bits
            subpackets_length = int(binary_str[7:22], 2)
            subpackets_str = binary_str[22:22+subpacket_length]
            payload = subpackets_str
            return (main_version, packet_type, length_type_id, payload)
        


In [6]:
def get_first_complete_packet(binary_str):
    main_version = binary_str[:3]
    packet_type = binary_str[3:6]
    if packet_type == '100':
        length_type_id = None
        payload = binary_str[6:]
        for i in range(len(payload)):
            if i % 5 == 0 and possible_payload[i] == '0':
                # if this bit is 0, this is the last group
                payload = payload[:i+5]
                full_packet = main_version + packet_type + payload
                return full_packet
    

In [39]:
"""
parsing a packet happens left to right: recursive thing, I think. The hard part really is the case where we know
how many packets are coming but not how long they are. The base case needs to be a literal packet (leaf).
But an operator can contain any number of operators, and really the thing here is finding all literal leaves.

for each packet, figure out the boundaries of the packets it contains.
if an operator packet contains any number of literal packets, find those.
the hard part is figuring out the boundaries of other operator packets

so for each packet, we need to figure out its children packets
if a child is a literal, that's easy
if a child is another operator, need to figure out that operator's boundaries

the main function takes a packet startpoint and figures out what index it ends at.
  if literal, that's easy.
  if operator, then drop into its subpackets, figure out what index they end at.
    first subpacket: figure out what index it ends at. if literal, that's easy.
    if operator, drop into its subpackets, figure out what index they end at.

"""

def find_end_index_for_packet(binary_str):
    packet_type = binary_str[3:6]
    if packet_type == '100':  # literal, base case
        payload = binary_str[6:]
        for i in range(len(payload)):
#             print('%d: %s' % (i, payload[i:]))
            if i % 5 == 0 and payload[i] == '0':
#                 print('last 4-digit number starts at position %d' % i)
                last_pos = i+11  # add another 5 bits for the last number, and 6 for starter padding
                return last_pos  # this is the last packet

packet = '110100101111111000101000'
end_index = find_end_index_for_packet(packet)
packet[:end_index]

'110100101111111000101'

In [24]:
'110100101111111000101000'[:20+1]

'110100101111111000101'